In [1]:
import pandas as pd
import uuid
from dataframe_sql import register_temp_table, query
import ast
import random
from tqdm import tqdm
import re

In [2]:
test_set = pd.read_json('wikisql_ann_test.json')

In [3]:
test_set.head()

,answer,question,id,header,rows,header_types
0,SELECT ` <col2> ` FROM ` table ` WHERE ` <col0...,<col0> Player : text <col1> No. : text <col2> ...,4e84108c351d47b5aedac962af6749cf,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
1,SELECT ` <col5> ` FROM ` table ` WHERE ` <col4...,<col0> Player : text <col1> No. : text <col2> ...,f6e1c2213a1d4cebbb1521da219c7aa0,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
2,SELECT ` <col5> ` FROM ` table ` WHERE ` <col4...,<col0> Player : text <col1> No. : text <col2> ...,1ce1252d4098425aaa9c38aeb5b79c69,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
3,SELECT COUNT ` <col5> ` FROM ` table ` WHERE `...,<col0> Player : text <col1> No. : text <col2> ...,33caff92db0442fa8870dee4ace70c4a,"['Player', 'No.', 'Nationality', 'Position', '...","[['Aleksandar Radojević', '25', 'Serbia', 'Cen...","['text', 'text', 'text', 'text', 'text', 'text']"
4,SELECT ` <col2> ` FROM ` table ` WHERE ` <col3...,<col0> No : real <col1> Date : text <col2> Rou...,acaa5428b2d541c7a7842de42e6fc77f,"['No', 'Date', 'Round', 'Circuit', 'Pole Posit...","[['1', '24 February', 'Qatar', 'Losail', 'Kevi...","['real', 'text', 'text', 'text', 'text', 'text..."


In [4]:
def register_table(query, data, row_id):
    tab_id = random.randint(0, 10000000)
    query = query.replace('` table `', f"table_{tab_id}")
    query = query.replace('` ','\'').replace(' `','\'')
    query = query.replace(' \'','\'')
    register_temp_table(data, f"table_{tab_id}")
    return query

In [5]:
answers = []
agg_operators = ['MAX', 'AVG', 'MIN', 'COUNT', 'SUM']
for idx, row in tqdm(test_set.iterrows()):
    answer = [[]]
    try:
        sql = row.answer
        question = row.question.split('</s>')[1]
        
        header = ast.literal_eval(row.header)
        header = [re.sub('[^A-Za-z0-9]+', '', col) for col in header]
        header_types = ast.literal_eval(row.header_types)
        rows = ast.literal_eval(row.rows)
        table = pd.DataFrame(rows, columns=header)
        for ix, column in enumerate(header):
            if f"` <col{ix}> `" in sql:
                if ' ' in column:
                    sql = sql.replace(f"` <col{ix}> `", f'\'{column}\'')
                else:
                    sql = sql.replace(f"` <col{ix}> `", column)
        for agg_operator in agg_operators:
            if agg_operator in sql:
                sql = sql.replace(agg_operator+' ', f' {agg_operator}(')
                sql = sql.replace(' FROM', f') FROM')
        for idx_, column_type in enumerate(header_types):
            if column_type == 'real':
                table[header[idx_]] = table[header[idx_]].astype(float)
                
        sql_f = register_table(query=sql, data=table, row_id=idx)
        for match in  re.findall('=\'[0-9.]+\'', sql_f):
            match_new = match.replace('\'','')
            sql_f = sql_f.replace(match, match_new)
        
        for op in  ['>=','=<','>','<']:
            if op in  sql_f:
                chars = '' 
                for char in sql_f.split(op)[1][1:]:
                    if char.isdigit() or char == '.':
                        chars += char 
                    else:
                        break
                sql_f = sql_f.replace(f"\'{chars}\'", chars)
        answers.append(query(sql_f).values)
    except Exception as e:
        answers.append(answer)
        continue

15878it [27:08,  9.75it/s]


In [14]:
test_set['answer_rows'] = answers

In [22]:
len(answers)

15878

In [27]:
counter = 0
data_set = []
for ix,row in test_set.iterrows():
    try:
        if len(answers[ix][0]):
            counter += 1
            print(row['answer_rows'])
            data_set.append(dict(row))
    except Exception as e:
        pass
counter

[['United States']]
[['Arkansas']]
[['Michigan']]
[[1]]
[['Netherlands']]
[[1]]
[['17 June']]
[[1]]
[['Canada']]
[[1]]
[[1]]
[[56.]]
[['Canada']]
[[77.]]
[['September 11, 1965']]
[['June 8, 1964']]
[['Group Captain']]
[['Colonel']]
[['Second Lieutenant']]
[['Air Vice-Marshal']]
[['Mayor']]
[['Roderick Strong']
 ['Davey Richards']
 ['Erick Stevens']]
[[1.]]
[[4.]]
[['119']]
[[8.]]
[['New Plymouth']]
[['Auckland']]
[[1]]
[[1]]
[['29–36']]
[[37.]]
[[1]]
[[1]]
[['News Radio 95 WXTK']]
[['AM 680 WRKO']]
[['News/Talk']]
[['Worcester, Massachusetts /116']]
[['1390 kHz']]
[['8.60p/call']]
[['4p/min']]
[[1]]
[[1]]
[['0843, 0844']
 ['0843, 0844']]
[['2.553p/min']]
[['/mẽra/']]
[[4.]]
[['O4 III']]
[[25.]]
[['66%']]
[[105579.]]
[[37767709.]]
[['Arizona Diamondbacks']]
[['27-23']]
[['Baseball']]
[['Pittsburgh Steelers']]
[['Carolina Panthers']]
[[1323.]]
[[637.]]
[[1]]
[[0.]]
[[1]]
[[1]]
[['Romantic Relationships']]
[['Istanbul']]
[[44000.]]
[[2014.]]
[[1]]
[['August22,2006']]
[['30%']]
[['800m-1,6

[[1]]
[['11']]
[[1]]
[[1]]
[[180.]]
[[1]]
[[0.]]
[[1]]
[[12.]]
[[13.]]
[[2]]
[['Snow']]
[['Right tackle']]
[['Redner']]
[[10.]]
[[25.]]
[[0.]]
[['Left end']]
[[0.]]
[['Albion (2 years)']]
[['Spec. Lit.']]
[[1]]
[['Redding, California']]
[["Law '04"]]
[[29.]]
[['Yes']]
[['Willie Heston']]
[[50.]]
[[63.]]
[['Curtis']]
[[1]]
[['Tom Hammond']]
[['Quarterback']]
[[2.]]
[[1]]
[[1.]]
[[5.]]
[[1]]
[[0.]]
[[10.]]
[['Yes']]
[[1]]
[[1]]
[[23.]]
[['Wednesdays at 10:00 PM']]
[[4074.]]
[[1]]
[[1]]
[['Fabian Cancellara']
 ['Mark Cavendish']]
[['Mark Cavendish']]
[['Alberto Contador']]
[['Brice Feillu']]
[[1]]
[[2]]
[[1]]
[[1]]
[[0.]]
[['Not filled this term']]
[[1]]
[['1:00pm']]
[[1]]
[['at Cincinnati Bengals']]
[[1]]
[[1.]]
[[3.]]
[['39.7']]
[[1]]
[['Columbia, South Carolina']]
[['Jr.']]
[['Mike Miller']]
[['Texas Arlington']]
[[11.]]
[['Running Back']]
[['870 AM']]
[[1]]
[['Mixtec Amuzgo Chatino']]
[['Enrique Ojeda ( CHI )']]
[[92.]]
[['Left Wing']]
[['Canada']]
[['Chris Ahrens']]
[['Defence']]
[['

[[1]]
[[1]]
[['March 15, 2002']]
[[0.]]
[[19.]]
[['United States']]
[['Bastogne']]
[['Panhard 70']]
[[1]]
[['Bastogne']]
[[1]]
[['Pierre de Caters']]
[[10.]]
[['80 × 32']]
[['Le Dernier Métro']]
[['Wild Reeds']]
[['Bertrand Tavernier Category:Articles with hCards']]
[['Black Orpheus']]
[['Nueva Ecija']]
[['Evicted']]
[['Unlimited']]
[['Unlimited']]
[[22.]]
[[1]]
[[1]]
[['Tim Mack']]
[['Matt Wilson']]
[[1]]
[['Matt Wilson']]
[[7395.]]
[['Southwark']]
[[8126.]]
[[8126.]]
[['2.35']]
[['West Philadelphia / Lower Merion Township']]
[['North Philadelphia']]
[[1]]
[['North Philadelphia']]
[['Private/Catholic']]
[['Dragons']]
[['92.3 News FM']]
[['88.7 News FM']]
[['10kW']]
[['101.5MHz']]
[['Bacolod']]
[['Metro Manila']]
[[21.]]
[['2.40.810']]
[['2.33.867']]
[[12.]]
[[1976.]]
[['MS']]
[[4.]]
[[3.]]
[[1]]
[[0.]]
[[1]]
[[0.]]
[['0']]
[['15.1']]
[['3rd, Northern']]
[['Did not qualify']]
[[2011.]]
[['Did not qualify']]
[[1996.]]
[[10.]]
[[1230.]]
[[13]]
[[1]]
[[15.]]
[['Oct 5, 2008']]
[[195000.]]


[['750,000 years']]
[[70.]]
[['Stony Brook']]
[['Richard Jefferson (19)']]
[['W 95–87 (OT)']]
[['@ Portland']]
[['S.D. Tex.']]
[['District of South Dakota']]
[['Western District of Virginia']]
[['20.35']]
[[1]]
[[1]]
[[12.]]
[['February 14, 2010']]
[['4th']]
[['M S Dhoni']]
[['2nd']]
[[2012.]]
[['19-11']]
[[1]]
[['.857']]
[['23-10']]
[[1]]
[[1]]
[['161.531mph']]
[['36.0']]
[[1997.]]
[[10.]]
[[1]]
[['Taii K. Austin']]
[[803.]]
[['"An Old Man Is Twice a Child"']]
[['12September2006']]
[['6 June 2009']]
[['27 June 2009']]
[['Steve Hughes Tom Stade Paul Tonkinson']]
[['1x05']]
[['Steve Hughes Tom Stade Paul Tonkinson']]
[[1]]
[[1]]
[[903.]]
[['October19,2009']]
[[15.]]
[[15.]]
[['LB']]
[[1]]
[['Weekly Charts']
 ['Monthly Charts (June 2009)']]
[[1]]
[[1.]]
[['Daily Charts']]
[['L 90–95 (OT)']]
[['Samuel Dalembert (11)']]
[['Dec. 29/08']]
[[52.]]
[['Dec. 7/08']]
[[1]]
[['March 7']]
[['L 84–98 (OT)']]
[[70.]]
[[71.]]
[['January 26']]
[['Slavi Zhekov']]
[[1]]
[['Ivelin Popov']]
[['Lokomotiv (P

[[11.]]
[[78.]]
[[6199.]]
[[3760.]]
[[3210.]]
[[1]]
[[1]]
[[1]]
[[0.]]
[[1]]
[[1]]
[['Nashville, Tennessee']]
[['Tigers']]
[['Fisk University']]
[[1]]
[['" Honky Tonk Women "']]
[["Auditioner's Choice"]]
[['Bottom 3']]
[['Unaired']]
[['1991']]
[['Benedictine College']]
[['rōmaji']]
[['governor']]
[[1]]
[['ふじさん']]
[['otya']]
[['May21,2011']]
[[1]]
[[7.]]
[[1]]
[[1]]
[[22974.]]
[['Golden Flashes']]
[[1]]
[['Adrian, MI']]
[[11.]]
[[1]]
[['Heather and Matt']]
[['4.8%']]
[['Danny and Frankie']]
[[5.]]
[['Monday-Friday 6:00pm']]
[[244.]]
[['Monday-Friday 6:00pm']]
[['Diva']]
[['Raw']]
[['Chavo Guerrero (Salvador Guerrero IV)']
 ['Cody Rhodes (Cody Runnels)']
 ['Chris Masters (Christopher Mordetzky)']
 ['Hornswoggle (Dylan Postl)']]
[[1.]]
[['Formula Three Sudamericana']]
[[1]]
[[1]]
[[1]]
[[1]]
[['86-71-91']]
[[1]]
[['Louisa Mazzurana']]
[[100.]]
[['Re-elected']]
[[2.]]
[[1]]
[['"Dis-Lodged"']]
[[1]]
[['Maria Fernanda Alvarez Teran']]
[[0.]]
[['Yes']]
[[1]]
[['Funky Lab Rat →']]
[[123.]]
[[1

...

[['Martyn Wilson']]
[['St. Louis Post Dispatch/ Research 2000']]
[[24.]]
[['Southern Outreach']]
[['Davos']]
[['1,000 m']]
[[0]]
[['12f']]
[[nan]]
[[314.]]
[['63,513']]
[[0.]]
[[10.]]
[['Christa Johnson , Jan Stephenson']]
[['Mark Skaife']]
[['October 13']]
[['11–16']]
[['" Everyday Is a Winding Road "']]
[[0.]]
[[nan]]
[['Granatelli Racing']]
[[8.]]
[[0.]]
[['Allen County War Memorial Coliseum']]
[[nan]]
[['Manchester Wolves']]
[[25.]]
[[nan]]
[[62.]]
[[0.]]
[['Tom Watson']]
[['19 October 2008']]
[['2-0']]
[['H']]
[['18,671']]
[[nan]]
[[0.]]
[[2.]]
[[0.]]
[[0.]]
[[nan]]
[[0]]
[[0]]
[[nan]]
[['480i']]
[[1.]]
[[0]]
[[71.]]
[[2034.]]
[['Cannot handle non-empty timestamp argument! 1933']]
[[nan]]
[[36.]]
[[nan]]
[['@ Philadelphia Flyers']]
[[nan]]
[[nan]]
[[0]]
[[nan]]
[[0]]
[[0.]]
[[32.]]
[['26,089']]
[['Philadelphia Eagles']]
[['Philadelphia Eagles']]
[['United States']]
[[nan]]
[['1:27.642']]
[['1:28.401']]
[['1:27.976']]
[[40.]]
[['Boston Garden']]
[['UK']]
[[nan]]
[[0]]
[[nan]]
[['Re

[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[['Scuderia Ferrari SpA SEFAC']]
[[0]]
[[nan]]
[[0]]
[['Michigan City, IN']]
[[1]]
[['3–1']]
[[0]]
[[nan]]
[['Götzis , Austria']]
[[nan]]
[[nan]]
[[0.]]
[[nan]]
[['1999/00']]
[['D41']]
[['2014']]
[[nan]]
[[3.]]
[[nan]]
[['Ricardo Zonta']]
[['Gearbox']]
[['Alexander Wurz']]
[[0.]]
[[nan]]
[[nan]]
[['Dianne Leighton, Kaye Whiteford']]
[['2008-07-08']]
[['Kathy Ireland Greens.com LPGA Classic']]
[[1]]
[[nan]]
[[nan]]
[[0.]]
[[0.]]
[[nan]]
[['August 19, 2008']]
[[0]]
[[0]]
[['October 2']]
[['Mercedes-Benz Superdome']]
[['Tampa Stadium']]
[[nan]]
[[0.]]
[[nan]]
[[0.]]
[['European Championships']]
[['Long jump']]
[['2008 Africa Cup of Nations']]
[[0]]
[['Chicago Bears']]
[[0.]]
[[nan]]
[[nan]]
[[9.029]]
[[nan]]
[[0.]]
[[0.]]
[[nan]]
[[nan]]
[['302.5']]
[['4 September 2013']]
[['2013 SAFF Championship']]
[['4 September 2013']]
[['Dasarath Rangasala Stadium, Kathmandu']]
[[0.]]
[[0]]
[['Williams - Supertec']]
[[0]]
[['Yamaha V10']]
[[0]]
[['1:00']]
[['Lo

[['Russell T Davies Mark Gatiss']]
[[nan]]
[['7 279']]
[['- 379']]
[[nan]]
[[0.]]
[[0.]]
[['United States']]
[['Canada']]
[[10.]]
[['Test Match']]
[['Wales']]
[['2-2']]
[['FC Bayern Munich - 1. FC Nuremberg']]
[['FC Bayern Munich - 1. FC Nuremberg']]
[[nan]]
[['Australia']]
[[36.]]
[[nan]]
[[0.]]
[[0]]
[['C']]
[['11 January 1563']]
[[nan]]
[['2 August 2012']]
[['24.07']]
[['USA Olympic Trials']]
[[0]]
[['2001']]
[[0]]
[[nan]]
[[8.]]
[['1963 , 1971 , 1973 1975 , 1980']
 ['1979']]
[[nan]]
[[1]]
[['16 January 1621']]
[['Five Nations']]
[[21.]]
[[nan]]
[['Australia']
 ['United States']
 ['United States']
 ['United States']
 ['United States']
 ['Fiji']
 ['United States']]
[[nan]]
[[1.]]
[[0.]]
[[25.]]
[['Five Nations']]
[['Spain']]
[['Sunk']]
[['550']]
[[14.]]
[['United States']]
[['0-1']]
[['Chicago Black Hawks']]
[[0.]]
[['Hong Kong (HKG)']]
[[nan]]
[[13.]]
[[nan]]
[[237.]]
[['–8']]
[[nan]]
[[8.]]
[[nan]]
[[nan]]
[[nan]]
[[1]]
[[7.]]
[[0.]]
[[nan]]
[[nan]]
[['0–3']]
[[0]]
[[nan]]
[[nan]]


 ['Win–Loss']]
[['Grand Slam Tournaments']
 ['Australian Open']
 ['French Open']
 ['Wimbledon']
 ['US Open']
 ['Win–Loss']]
[[nan]]
[['4th']]
[[0.]]
[[nan]]
[[2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]]
[[2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]
 [2000]]
[[2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]
 [2001]]
[[2001]]
[[2009]
 [2009]
 [2009]
 [2009]
 [2009]
 [2009]]
[[5.]]
[['BRP 1']
 ['BRP 1']]
[[6.]]
[[1]]
[[nan]]
[['1-2']]
[['19 December 2007']
 ['20 December 2007']
 ['21 December 2007']
 ['22 December 2007']]
[['18 December 2007']]
[[0.]]
[[0]]
[['Bangkok , Thailand']]
[[nan]]
[[0]]
[['du, do']]
[['pandž']]
[['sau']]
[['nuh, noh']]
[['deš']]
[['lui']]
[['Hard']]
[[nan]]
[['1A1']]
[[360.]]
[[0]]
[[nan]]
[[0.]]
[['Frank Williams Racing Cars']
 ['Frank Williams Racing Cars']
 ['T

6779

In [29]:
pd.DataFrame(data_set).shape#.to_csv('wikisql_ann_test_tapas.csv', index=False,encoding='utf-8-sig')

(6779, 7)